In [21]:
import pandas as pd
import numpy as np

# Load the data
file_path = 'data.xlsx'
data = pd.read_excel(file_path)

# Transpose data to get time series per user
data.set_index(['Nama Pemakai', 'Kategori', 'Lokasi', 'Daya Tersambung (VA)'], inplace=True)
data = data.transpose()

# Check for missing values and handle them
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)

# Dummy data for future use (if needed)
dummy_data = pd.DataFrame(np.random.rand(52, data.shape[1]), columns=data.columns)

# Display the prepared data
data.head()


C:\Users\User\AppData\Local\Temp\ipykernel_4896\1597903007.py:13: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\User\AppData\Local\Temp\ipykernel_4896\1597903007.py:14: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Nama Pemakai,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,...,User_41,User_42,User_43,User_44,User_45,User_46,User_47,User_48,User_49,User_50
Kategori,Rumah Tangga,Sosial,Lain-lain,Lain-lain,Lain-lain,Bisnis,Lain-lain,Industri,Rumah Tangga,Sosial,...,Industri,Rumah Tangga,Industri,Sosial,Sosial,Rumah Tangga,Sosial,Rumah Tangga,Bisnis,Rumah Tangga
Lokasi,Muara Batu,Blang Mangat,Blang Mangat,Lhokseumawe,Muara Batu,Muara Batu,Muara Batu,Muara Batu,Syamtalira Bayu,Syamtalira Bayu,...,Baktiya,Lhokseumawe,Muara Batu,Blang Mangat,Kuta Makmur,Blang Mangat,Lhokseumawe,Baktiya,Syamtalira Bayu,Blang Mangat
Daya Tersambung (VA),1473,1368,2523,3465,701,5431,2918,14651,3955,1023,...,13091,4412,11434,1450,2272,680,1323,4354,7845,2687
Minggu_1,136,221,159,381,234,828,202,1413,199,210,...,1216,431,1024,167,103,284,135,264,855,82
Minggu_2,61,204,262,188,232,721,175,1156,161,102,...,1232,305,1036,100,134,247,226,487,554,130
Minggu_3,496,236,239,179,259,668,341,1376,397,268,...,1198,324,1348,143,183,483,141,143,474,455
Minggu_4,251,189,379,78,259,917,492,1384,113,116,...,1022,277,1378,183,235,111,241,55,556,186
Minggu_5,257,281,189,54,473,648,479,1282,124,152,...,1167,186,1013,235,122,385,108,277,509,110


In [37]:
from flask import Flask, jsonify, request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

app = Flask(__name__)

class VARMA:
    def __init__(self, p, q):
        self.p = p
        self.q = q
        self.phi = None
        self.theta = None
        self.mu = None
    
    def fit(self, Y):
        T, k = Y.shape
        self.mu = Y.mean(axis=0)
        Y = Y - self.mu
        
        self.phi = np.zeros((self.p, k, k))
        self.theta = np.zeros((self.q, k, k))
        
        residuals = np.zeros((T, k))
        
        for t in range(max(self.p, self.q), T):
            y_t = Y[t]
            y_past = [Y[t-i-1] for i in range(self.p)]
            residuals_t = [residuals[t-i-1] for i in range(self.q)]
            
            y_hat = np.zeros(k)
            
            for i in range(self.p):
                y_hat += np.dot(self.phi[i], y_past[i])
            
            for i in range(self.q):
                y_hat += np.dot(self.theta[i], residuals_t[i])
            
            residuals[t] = y_t - y_hat
        
        for i in range(self.p):
            X = np.vstack([Y[t-i-1] for t in range(max(self.p, self.q), T)])
            self.phi[i] = np.linalg.lstsq(X, Y[max(self.p, self.q):], rcond=None)[0].T
        
        for i in range(self.q):
            X = np.vstack([residuals[t-i-1] for t in range(max(self.p, self.q), T)])
            self.theta[i] = np.linalg.lstsq(X, residuals[max(self.p, self.q):], rcond=None)[0].T
    
    def predict(self, Y, steps):
        T, k = Y.shape
        Y = Y - self.mu
        predictions = np.zeros((steps, k))
        residuals = np.zeros((T, k))
        
        for t in range(max(self.p, self.q), T):
            y_t = Y[t]
            y_past = [Y[t-i-1] for i in range(self.p)]
            residuals_t = [residuals[t-i-1] for i in range(self.q)]
            
            y_hat = np.zeros(k)
            
            for i in range(self.p):
                y_hat += np.dot(self.phi[i], y_past[i])
            
            for i in range(self.q):
                y_hat += np.dot(self.theta[i], residuals_t[i])
            
            residuals[t] = y_t - y_hat
        
        for s in range(steps):
            y_past = [Y[-i-1] for i in range(self.p)]
            residuals_t = [residuals[-i-1] for i in range(self.q)]
            
            y_hat = np.zeros(k)
            
            for i in range(self.p):
                y_hat += np.dot(self.phi[i], y_past[i])
            
            for i in range(self.q):
                y_hat += np.dot(self.theta[i], residuals_t[i])
            
            predictions[s] = y_hat
            Y = np.vstack([Y, y_hat])
        
        return predictions + self.mu

@app.route('/predict', methods=['GET'])
def predict():
    file_path = 'data.xlsx'
    if not os.path.exists(file_path):
        return jsonify({"status": "error", "message": "Data file not found"}), 404

    data = pd.read_excel(file_path)
    data.set_index(['Nama Pemakai', 'Kategori', 'Lokasi', 'Daya Tersambung (VA)'], inplace=True)
    data = data.transpose()

    # Fit the VARMA model
    varma_model = VARMA(p=1, q=1)
    varma_model.fit(data.values)

    # Predict the next 10 weeks
    future_steps = 10
    predictions = varma_model.predict(data.values, steps=future_steps)

    # Create a DataFrame for predictions and round to four decimal places
    predicted_df = pd.DataFrame(predictions, columns=data.columns).round(4)

    # Combine actual usage and predictions in a single DataFrame
    combined_df = pd.DataFrame()

    for user in data.columns:
        user_data = data[user]
        user_predictions = predicted_df[user]

        # Combine actual usage and predictions in a single row
        user_row = pd.Series([user], index=['User'])
        usage_weeks = [f'Week_{i+1}' for i in range(len(user_data))]
        prediction_weeks = [f'Pred_Week_{i+1}' for i in range(future_steps)]

        for week, usage in zip(usage_weeks, user_data):
            user_row[week] = round(usage, 4)

        for week, prediction in zip(prediction_weeks, user_predictions):
            user_row[week] = round(prediction, 4)

        # Append to combined DataFrame
        combined_df = pd.concat([combined_df, user_row.to_frame().T], ignore_index=True)

        # Plot the data
        plt.figure()
        plt.plot(range(1, len(user_data) + 1), user_data, label='Actual Usage')
        plt.plot(range(len(user_data) + 1, len(user_data) + future_steps + 1), user_predictions, label='Predicted Usage', linestyle='--')
        plt.xlabel('Week')
        plt.ylabel('Electricity Usage')
        plt.title(f'Usage Prediction for {user}')
        plt.legend()
        plt.savefig(f'{user}_prediction.png')
        plt.close()

    # Convert combined DataFrame to JSON
    combined_json = combined_df.to_dict(orient='records')

    return jsonify({"status": "success", "data": combined_json})

if __name__ == '__main__':
    app.run(debug=True)


Predictions saved to CSV and visualizations created.
